# Transport Model (Phase 1)

### Phase 1 model overview

![image info](./data/transport_model_v1/phase-1-data-flow.png)

In the Phase 1 model, `FASTSim` is used to estimate the vehicle design and operations exogenously to `Tyche`. The input data (including vehicle specifications, expert elicitation data on technology cost/performance) is used as inputs into `FASTSim`. FASTSim then (1) sizes the powertrain (fuel converter power to meet acceleration requirements, fuel/energy storage to meet range requirements), and (2) estimates the fuel efficiency of the vehicle. This data is all input into the `Tyche` data files, ensuring the input data to `FASTSim` and `Tyche` are the same for each `Tyche` case. 

This modeling architecture allows for detailed powertrain dynamics to be represented in `FASTSim` rather than in `Tyche` through explicit modeling or look-up tables (which are proposed to be used in Phase 2, see below). However, probabilistic information on technology component performance (e.g., diesel engine efficiency improvements based on expert elicitations) are harder to represent in this framework since the powertrain sizing depends on the performance of the components. However, this framework allows for component cost data to be represented probabilistically within `Tyche` in a straight-forward manner.

### Phase 2 model overview

![image info](./data/transport_model_v1/phase-2-data-flow.png)

In Phase 2, a more complex modeling architecture will be used. An ensemble of `FASTSim` runs could provide a look-up table for `Tyche` to use endogenously, allowing for `Tyche` to size the fuel converter power, fuel/energy storage, and fuel efficiency of the vehicle. This approach effectively allows for vehicle operations to be estimated endogenously within `Tyche` without fully developing the `FASTSim` code in the `Tyche` framework.

This architecture allows for probabilistic data to apply to *both* the technology component cost and performance data. This expands the usefulness of the `Tyche` framework to allow for expert elicitation data on technology component performance (e.g., fuel converter efficiency) in addition to technology component costs.

## Running the model.

### Create the Conda environment and launch the Jupyter Notebook from the Anaconda Prompt Command Line.

```
conda env create --file conda\win.yml
conda activate tyche
pip install mip
```

#### Activate the Tyche environment in Anaconda Prompt.

```
conda activate tyche
```

#### Launch jupyter notebook from Anaconda Prompt to use activated virtual environment.

```
jupyter notebook
```

## Set up.

### Import packages.

#### Import the system packages.

In [1]:
import os
import sys

#### Add the main Tyche packages to the search path.

In [2]:
sys.path.insert(1, os.path.abspath("../src"))

#### Import tyche and related packages.

In [3]:
import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb
import tyche             as ty

## Design the technology model.

### Technology name.

Choose a unique name for the technology being modeled. *In this example we'll call the technology `Class 8 Diesel Tractor`.*

Also choose a unique name for the default, reference, or base-case scenario. *In this example, we'll call the default scenario `Reference`.*

### Dimensions and indices.

Decide which quantities will be tracked as indices in the model, and settle on their units of measurment and default values:
1.  Types of capital costs. *In this example we use FASTSim's capital cost categories . . .*
    *   Glider
    *   Fuel converter
    *   Fuel storage
    *   Battery
    *   Electric drive train
    *   Plug
2.  Types of fixed cost. *In this example we have . . .*
    *   Maintanence and repair
    *   Dwell time (downtime for refueling/recharging)
    *   Carbon price
3.  Inputs to the process. *In this example we have . . .*
    *   Fuel
4.  Outputs from the process. *In this example we have . . .*
    *   VMT
5.  Metrics. *In this example we have . . .*
    *   LCOD
    *   MSPR
    *   Lifetime cost
    *   Vehicle weight
    *   Energy consumption 
    *   GHG emissions
6.  Parameters. *In this example we have . . .*
    *   Fuel converter size/weight/cost
    *   Fuel storage (tank) size/weight/cost
    *   Energy storage (battery) size/weight/cost
    *   Electric drive size/weight/cost
    *   Dwell time cost and refueling rate
    *   Carbon prices

### View the `indices` table.

In [4]:
transport_designs = ty.Designs("data/transport_model_v1")
transport_designs.indices.reset_index("Index").sort_values(["Type", "Offset"])

Index  Offset  \
Technology             Type                                  
Class 8 Diesel Tractor Capital              Glider       0   
                       Capital      Fuel converter       1   
                       Capital        Fuel storage       2   
                       Capital             Battery       3   
                       Capital      Electric drive       4   
                       Capital                Plug       5   
                       Fixed     Other Fixed Costs       0   
                       Input                  Fuel       0   
                       Metric                 MSRP       0   
                       Metric        Lifetime cost       1   
                       Metric       Vehicle weight       2   
                       Metric   Energy consumption       3   
                       Metric        GHG emissions       4   
                       Output                  VMT       0   

                                                                      Description  \
Technology             Type                                                         
Class 8 Diesel Tractor Capital                                     Vehicle glider   
                       Capital   Vehicle fuel converter (e.g., engine, fuel cell)   
                       Capital  Vehicle fuel storage (e.g., gasoline tank, hyd...   
                       Capital                                    Vehicle battery   
                       Capital     Vehicle power electronics and electric machine   
                       Capital                     Vehicle plug to charge battery   
                       Fixed    Other fixed costs (e.g., insurance, permits, t...   
                       Input    Fuel/energy input (e.g., diesel, electricity, ...   
                       Metric             MSRP = Minimimum suggested retail price   
                       Metric                                                       
                       Metric                                                       
                       Metric                                                       
                       Metric                                                       
                       Output                                     Miles travelled   

                               Notes  
Technology             Type           
Class 8 Diesel Tractor Capital        
                       Capital        
                       Capital        
                       Capital        
                       Capital        
                       Capital        
                       Fixed          
                       Input          
                       Metric         
                       Metric         
                       Metric         
                       Metric         
                       Metric         
                       Output

### View the `results` table.

In [5]:
transport_designs.results

Units  \
Technology             Variable Index                         
Class 8 Diesel Tractor Cost     LCOD                 $/mile   
                       Metric   Energy consumption      gge   
                                GHG emissions        g-CO2e   
                                Lifetime cost             $   
                                MSRP                      $   
                                Vehicle weight           kg   
                       Output   VMT                 mile/yr   

                                                                                      Notes  
Technology             Variable Index                                                        
Class 8 Diesel Tractor Cost     LCOD                       LCOD = levelized cost of driving  
                       Metric   Energy consumption                                           
                                GHG emissions                                                
                                Lifetime cost                                                
                                MSRP                MSRP = Minimimum suggested retail price  
                                Vehicle weight                                               
                       Output   VMT                  Yearly vehicle miles travelled (scale)

### View the `designs` table.

In [6]:
transport_designs.designs.reset_index(["Variable", "Index"]).sort_values(["Variable", "Index"])

Variable           Index  \
Technology             Scenario                                       
Class 8 Diesel Tractor Reference              Input            Fuel   
                       Reference   Input efficiency            Fuel   
                       Reference        Input price            Fuel   
                       Reference           Lifetime         Battery   
                       Reference           Lifetime  Electric drive   
                       Reference           Lifetime  Fuel converter   
                       Reference           Lifetime    Fuel storage   
                       Reference           Lifetime          Glider   
                       Reference           Lifetime            Plug   
                       Reference  Output efficiency             VMT   
                       Reference       Output price             VMT   
                       Reference              Scale             VMT   

                                                Value      Units  \
Technology             Scenario                                    
Class 8 Diesel Tractor Reference                  1.0        gge   
                       Reference  0.12575452716297789   gge/mile   
                       Reference   3.9759999999999995      $/gge   
                       Reference                 10.0       year   
                       Reference                 10.0       year   
                       Reference                 10.0       year   
                       Reference                 10.0       year   
                       Reference                 10.0       year   
                       Reference                 10.0       year   
                       Reference                  1.0  mile/mile   
                       Reference                  0.0       $/mi   
                       Reference             100000.0  mile/year   

                                                                          Notes  
Technology             Scenario                                                  
Class 8 Diesel Tractor Reference                     Gasoline gallon equivalent  
                       Reference   Inverse of diesel fuel economy of 7 mile/dge  
                       Reference             Equates to a $3.5/gal diesel price  
                       Reference                          Lifetime of component  
                       Reference                          Lifetime of component  
                       Reference                          Lifetime of component  
                       Reference                          Lifetime of component  
                       Reference                          Lifetime of component  
                       Reference                          Lifetime of component  
                       Reference     Placeholder varible since output is needed  
                       Reference     Placeholder varible since output is needed  
                       Reference  Average yearly VMT for Class 8 diesel tractor

### View the `parameters` table.

In [7]:
transport_designs.parameters.reset_index("Parameter").sort_values("Offset")

Parameter  Offset  \
Technology             Scenario                                             
Class 8 Diesel Tractor Reference                      Glider cost       0   
                       Reference        Fuel converter cost slope       1   
                       Reference        Fuel converter fixed cost       2   
                       Reference          Fuel storage cost slope       3   
                       Reference          Fuel storage fixed cost       4   
                       Reference        Energy storage cost slope       5   
                       Reference        Energy storage fixed cost       6   
                       Reference  Energy storage power cost slope       7   
                       Reference        Electric drive cost slope       8   
                       Reference        Electric drive fixed cost       9   
                       Reference                        Plug cost      10   
                       Reference                    Glider weight      11   
                       Reference              Fuel converter size      12   
                       Reference            Fuel converter weight      13   
                       Reference                Fuel storage size      14   
                       Reference              Fuel storage weight      15   
                       Reference              Energy storage size      16   
                       Reference            Energy storage weight      17   
                       Reference             Energy storage power      18   
                       Reference              Electric drive size      19   
                       Reference            Electric drive weight      20   
                       Reference                  Dwell time cost      21   
                       Reference             Dwell refueling rate      22   
                       Reference                 Dwell time basis      23   
                       Reference               Dwell time boolean      24   
                       Reference          Maintance & repair cost      25   
                       Reference                 Other fixed cost      26   
                       Reference      Cost to price markup factor      27   
                       Reference            Fuel carbon intensity      28   
                       Reference                      Carbon cost      29   
                       Reference              Carbon cost boolean      30   
                       Reference            Input fuel efficiency      31   
                       Reference                 Tractor lifetime      32   

                                                Value       Units  \
Technology             Scenario                                     
Class 8 Diesel Tractor Reference    82333.33333333333           $   
                       Reference                 50.0        $/kW   
                       Reference               5000.0           $   
                       Reference                 0.07       $/kWh   
                       Reference                  0.0           $   
                       Reference                  0.0       $/kWh   
                       Reference                  0.0           $   
                       Reference                  0.0        $/kW   
                       Reference                  0.0        $/kW   
                       Reference                  0.0           $   
                       Reference                  0.0           $   
                       Reference              14000.0          kg   
                       Reference                317.0          kW   
                       Reference               1385.0          kg   
                       Reference               3400.0         kWh   
                       Reference                415.0          kg   
                       Reference                  0.0         kWh   
                       Reference                 

## The transport technology model.

The implementation of a technology model consists of a capital cost function, a fixed cost function, a production function, and a metrics function.

The (Phase 1) transport model is located at [technology/transport_model.py](./technology/transport_model.py)

See the [Tyche model](https://tyche.live/doc-src/formulation.html) documentation for more details on model formulation.

### View the `functions` table.

In [8]:
transport_designs.functions

,Style,Module,Capital,Fixed,Production,Metrics,Notes
Technology,,,,,,,
Class 8 Diesel Tractor,numpy,transport_model,capital_cost,fixed_cost,production,metrics,


## Create investment for reference case.

### View the `tranches` table.

In [9]:
transport_investments = ty.Investments("data/transport_model_v1")
transport_investments.tranches

,,,Amount,Notes
Category,Tranche,Scenario,,
No Category,Business as Usual,Reference,0.0,Baseline scenario


### View the `investments` table.

In [10]:
transport_investments.investments

,,,Notes
Investment,Category,Tranche,
No Investment,No Category,Business as Usual,


## Simulate the base case or reference scenario.

### Load the data.

In [11]:
transport_designs     = ty.Designs(    "data/transport_model_v1")
transport_investments = ty.Investments("data/transport_model_v1")

### Compile the production and metric functions for each technology in the dataset.

In [12]:
transport_designs.compile()

### Compute costs and metrics for tranches.

Tranches are atomic units for building investment portfolios. Evaluate all of the tranches, so we can assemble them into investments (portfolios).

In [13]:
tranche_results = transport_investments.evaluate_tranches(transport_designs, sample_count=4)

The tranche amounts are simple how much each tranche costs.

In [14]:
tranche_results.amounts

,,Amount
Category,Tranche,
No Category,Business as Usual,0.0


The tranch metrics show the values of each metric for each member of the ensemble of simulations.

In [15]:
tranche_results.metrics

Value  \
Category    Tranche           Scenario  Sample Technology             Index                              
No Category Business as Usual Reference 1.0    Class 8 Diesel Tractor Energy consumption  1.257545e+05   
                                                                      GHG emissions       1.126920e+09   
                                                                      Lifetime cost       1.617758e+06   
                                                                      MSRP                1.551320e+05   
                                                                      Vehicle weight      1.580000e+04   
                                        2.0    Class 8 Diesel Tractor Energy consumption  1.257545e+05   
                                                                      GHG emissions       1.126920e+09   
                                                                      Lifetime cost       1.617758e+06   
                                                                      MSRP                1.551320e+05   
                                                                      Vehicle weight      1.580000e+04   
                                        3.0    Class 8 Diesel Tractor Energy consumption  1.257545e+05   
                                                                      GHG emissions       1.126920e+09   
                                                                      Lifetime cost       1.617758e+06   
                                                                      MSRP                1.551320e+05   
                                                                      Vehicle weight      1.580000e+04   
                                        4.0    Class 8 Diesel Tractor Energy consumption  1.257545e+05   
                                                                      GHG emissions       1.126920e+09   
                                                                      Lifetime cost       1.617758e+06   
                                                                      MSRP                1.551320e+05   
                                                                      Vehicle weight      1.580000e+04   

                                                                                           Units  
Category    Tranche           Scenario  Sample Technology             Index                       
No Category Business as Usual Reference 1.0    Class 8 Diesel Tractor Energy consumption     gge  
                                                                      GHG emissions       g-CO2e  
                                                                      Lifetime cost            $  
                                                                      MSRP                     $  
                                                                      Vehicle weight          kg  
                                        2.0    Class 8 Diesel Tractor Energy consumption     gge  
                                                                      GHG emissions       g-CO2e  
                                                                      Lifetime cost            $  
                                                                      MSRP                     $  
                                                                      Vehicle weight          kg  
                                        3.0    Class 8 Diesel Tractor Energy consumption     gge  
                                                                      GHG emissions       g-CO2e  
                                                                      Lifetime cost            $  
                                                                      MSRP                     $  
                                                                      Vehicle weight          kg  
                                        4.0    Class 8 Diesel Tractor Energy cons

In [16]:
tranche_results.metrics.groupby("Index").aggregate(np.median)

,Value
Index,
Energy consumption,1.257545e+05
GHG emissions,1.126920e+09
Lifetime cost,1.617758e+06
MSRP,1.551320e+05
Vehicle weight,1.580000e+04


### Compute costs and metrics for investments.

Now evaluate the investments.

In [17]:
investment_results = transport_investments.evaluate_investments(transport_designs, sample_count=5)

The investment amounts are simple how much each tranche costs.

In [18]:
investment_results.amounts

,Amount
Investment,
No Investment,0.0


The investment metrics show the values of each metric for each member of the ensemble of simulations.

In [19]:
investment_results.metrics

Value  \
Investment    Category    Tranche           Scenario  Sample Technology             Index                              
No Investment No Category Business as Usual Reference 1.0    Class 8 Diesel Tractor Energy consumption  1.257545e+05   
                                                                                    GHG emissions       1.126920e+09   
                                                                                    Lifetime cost       1.617758e+06   
                                                                                    MSRP                1.551320e+05   
                                                                                    Vehicle weight      1.580000e+04   
                                                      2.0    Class 8 Diesel Tractor Energy consumption  1.257545e+05   
                                                                                    GHG emissions       1.126920e+09   
                                                                                    Lifetime cost       1.617758e+06   
                                                                                    MSRP                1.551320e+05   
                                                                                    Vehicle weight      1.580000e+04   
                                                      3.0    Class 8 Diesel Tractor Energy consumption  1.257545e+05   
                                                                                    GHG emissions       1.126920e+09   
                                                                                    Lifetime cost       1.617758e+06   
                                                                                    MSRP                1.551320e+05   
                                                                                    Vehicle weight      1.580000e+04   
                                                      4.0    Class 8 Diesel Tractor Energy consumption  1.257545e+05   
                                                                                    GHG emissions       1.126920e+09   
                                                                                    Lifetime cost       1.617758e+06   
                                                                                    MSRP                1.551320e+05   
                                                                                    Vehicle weight      1.580000e+04   
                                                      5.0    Class 8 Diesel Tractor Energy consumption  1.257545e+05   
                                                                                    GHG emissions       1.126920e+09   
                                                                                    Lifetime cost       1.617758e+06   
                                                                                    MSRP                1.551320e+05   
                                                                                    Vehicle weight      1.580000e+04   

                                                                                                         Units  
Investment    Category    Tranche           Scenario  Sample Technology             Index                       
No Investment No Category Business as Usual Reference 1.0    Class 8 Diesel Tractor Energy consumption     gge  
                                                                                    GHG emissions       g-CO2e  
                                                                                    Lifetime cost            $  
                                                                                    MSRP                     $  
                                                                                    Vehicle weight          kg  
                                                      2.0    Class 8 Diesel Tra

A summary of the metrics is available at the investment level rather than at the tranche level.

In [20]:
investment_results.summary

Value   Units
Investment    Sample Index                                   
No Investment 1.0    Energy consumption  1.257545e+05     gge
                     GHG emissions       1.126920e+09  g-CO2e
                     Lifetime cost       1.617758e+06       $
                     MSRP                1.551320e+05       $
                     Vehicle weight      1.580000e+04      kg
              2.0    Energy consumption  1.257545e+05     gge
                     GHG emissions       1.126920e+09  g-CO2e
                     Lifetime cost       1.617758e+06       $
                     MSRP                1.551320e+05       $
                     Vehicle weight      1.580000e+04      kg
              3.0    Energy consumption  1.257545e+05     gge
                     GHG emissions       1.126920e+09  g-CO2e
                     Lifetime cost       1.617758e+06       $
                     MSRP                1.551320e+05       $
                     Vehicle weight      1.580000e+04      kg
              4.0    Energy consumption  1.257545e+05     gge
                     GHG emissions       1.126920e+09  g-CO2e
                     Lifetime cost       1.617758e+06       $
                     MSRP                1.551320e+05       $
                     Vehicle weight      1.580000e+04      kg
              5.0    Energy consumption  1.257545e+05     gge
                     GHG emissions       1.126920e+09  g-CO2e
                     Lifetime cost       1.617758e+06       $
                     MSRP                1.551320e+05       $
                     Vehicle weight      1.580000e+04      kg